In [1]:
from cpsat_autotune import (
    tune_for_quality_within_timelimit,
    import_model,
    tune_time_to_optimal,
)

In [2]:
model = import_model("models/model_proto_-1368620059351071685.pb")

In [3]:
# tune_for_quality_within_timelimit(models[0], 3, 10, "minimize", n_samples_per_param=2, max_samples_per_param=5, n_trials=100)
best = tune_time_to_optimal(
    model,
    max_time_in_seconds=3,  # Enter a time limit slightly above what the solver with default parameters needs
    n_samples_for_trial=5,  # Number of samples for each trial
    n_samples_for_verification=20,  # Number of samples for each statistically relevant comparison.
    n_trials=50,  # Number of trials to run with Optuna
)

[I 2024-08-16 18:03:13,951] A new study created in memory with name: no-name-0202c6cf-c3ef-4ca9-b692-641a43db037d


Baseline: min=1.125892, mean=1.6253635, max=2.15166


[I 2024-08-16 18:03:20,855] Trial 0 finished with value: 1.3803072 and parameters: {'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'also_bump_variables_in_conflict_reasons': False, 'binary_minimization_algorithm': 1, 'clause_cleanup_protection': 0, 'presolve_bve_threshold': 1, 'max_presolve_iterations': 2, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 4, 'encode_complex_linear_constraint_with_integer': False, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:probing': False, 'search_branching': 0, 'repair_hint': False, 'use_lb_relax_lns': False, 'use_objective_lb_search': False, 'use_objective_shaving_search': False, 'optimi

Returning knockout result


[I 2024-08-16 18:03:26,907] Trial 2 finished with value: 30.0 and parameters: {'preferred_variable_order': 1, 'use_erwa_heuristic': True, 'also_bump_variables_in_conflict_reasons': False, 'binary_minimization_algorithm': 1, 'clause_cleanup_protection': 0, 'presolve_bve_threshold': 2, 'max_presolve_iterations': 1, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 3, 'encode_complex_linear_constraint_with_integer': True, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': True, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:probing': True, 'search_branching': 8, 'repair_hint': True, 'use_lb_relax_lns': True, 'use_objective_lb_search': False, 'use_objective_shaving_search': False, 'optimize_with_cor

...

1. preferred_variable_order Sets the initial order in which variables are selected during the search. The options  
are:                                                                                                               

 • 0 (IN_ORDER): Select variables in the order they appear in the problem definition.                              
 • 1 (IN_REVERSE_ORDER): Select variables in reverse order.                                                        
 • 2 (IN_RANDOM_ORDER): Select variables randomly.                                                                 

The order of variable selection can have a significant impact on the efficiency of the search, though, this is only
the initial order and the solver will actually try to learn a better order. The search will restart frequently such
that this initial order should only influence the beginning of the search.                                         

2. cp_model_probing_level Defines the intensity of probing during presolve, where variables are temporarily fixed  
to infer more information about the problem. Higher levels of probing can result in a more simplified problem but  
require more computation time during presolve.                                                                     

3. presolve_probing_deterministic_time_limit Sets a deterministic time limit for probing during presolve. This     
parameter ensures that the presolve phase does not consume too much time, allowing the solver to proceed to the    
main search phase in a timely manner.                                                                              

4. search_branching Defines the branching strategy the solver uses to navigate the search tree. The options are:   

 • 0 (AUTOMATIC_SEARCH): The solver automatically selects the most appropriate strategy.                           
 • 1 (FIXED_SEARCH): Follows a fixed variable order, as specified by the user or the problem model.                
 • 2 (PORTFOLIO_SEARCH): Uses a combination of multiple strategies to explore the search space.                    
 • 3 (LP_SEARCH): Branches based on the LP relaxation of the problem, leveraging the reduced costs of variables.   
 • 4 (PSEUDO_COST_SEARCH): Branches using pseudo-costs, which are estimates of the impact of branching decisions   
   based on past experiences.                                                                                      
 • 5 (PORTFOLIO_WITH_QUICK_RESTART_SEARCH): Quickly explores different heuristics with low conflict limits, aiming 
   to find a good initial solution.                                                                                
 • 6 (HINT_SEARCH): Prioritizes decisions based on hints provided by the user or the problem model.                
 • 7 (PARTIAL_FIXED_SEARCH): Begins with a fixed strategy, then switches to automatic search for the remaining     
   decisions.                                                                                                      
 • 8 (RANDOMIZED_SEARCH): Introduces randomization into branching decisions to diversify the search.               

5. cut_level Sets the level of effort the solver will invest in generating cutting planes, which are linear        
constraints added to remove infeasible regions. Properly applied, cuts can significantly reduce the search space   
and help the solver find an optimal solution more quickly.

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━┓
┃ Metric                 ┃ Mean ┃  Min ┃  Max ┃ #Samples ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━┩
│ Default Metric Value   │ 1.63 │ 1.13 │ 2.15 │       10 │
│ Optimized Metric Value │ 0.53 │ 0.44 │ 0.61 │       10 │
└────────────────────────┴──────┴──────┴──────┴──────────┘

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────────────── WARNING ────────────────────────────────────────────────────╮
│                                                                                                                 │
│      The optimized parameters listed above were obtained based on a sampling approach                           │
│      and may not fully capture the complexities of the entire problem space.                                    │
│      While statistical reasoning has been applied, these results should be considered                           │
│      as a suggestion for further evaluation rather than definitive settings.                                    │
│                                                                                                                 │
│      It is strongly recommended to validate these parameters in larger, more comprehensive                      │
│      experiments before adopting them in critical applications.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [4]:
best = tune_for_quality_within_timelimit(
    model,
    max_time_in_seconds=0.5,
    obj_for_timeout=0,
    direction="maximize",
    n_samples_for_trial=5,  # Number of samples for each trial
    n_samples_for_verification=20,  # Number of samples for each statistically relevant comparison.
    n_trials=50,  # Number of trials to run with Optuna
)

[I 2024-08-16 18:08:04,091] A new study created in memory with name: no-name-3086a258-e407-4181-a39a-73d7ec8d8a3c


Baseline: min=130872.0, mean=131935.0, max=132805.0


[I 2024-08-16 18:08:06,173] Trial 0 finished with value: 130283.0 and parameters: {'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'also_bump_variables_in_conflict_reasons': False, 'binary_minimization_algorithm': 1, 'clause_cleanup_protection': 0, 'presolve_bve_threshold': 1, 'max_presolve_iterations': 2, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 4, 'encode_complex_linear_constraint_with_integer': False, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:probing': False, 'search_branching': 0, 'repair_hint': False, 'use_lns_only': False, 'use_lb_relax_lns': False, 'use_objective_lb_search': False, 'use_objective_shaving_s

Returning knockout result


[I 2024-08-16 18:08:08,735] Trial 1 finished with value: 132142.8 and parameters: {'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'also_bump_variables_in_conflict_reasons': True, 'binary_minimization_algorithm': 3, 'clause_cleanup_protection': 0, 'presolve_bve_threshold': 2, 'max_presolve_iterations': 2, 'cp_model_probing_level': 0, 'presolve_probing_deterministic_time_limit': 3, 'encode_complex_linear_constraint_with_integer': True, 'ignore_subsolvers:default_lp': True, 'ignore_subsolvers:fixed': True, 'ignore_subsolvers:no_lp': True, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': True, 'ignore_subsolvers:reduced_costs': True, 'ignore_subsolvers:quick_restart': True, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:probing': True, 'search_branching': 6, 'repair_hint': True, 'use_lns_only': True, 'use_lb_relax_lns': False, 'use_objective_lb_search': False, 'use_objective_shaving_search': Fal

...

1. clause_cleanup_protection Specifies the level of protection against clause cleanup. The options are:            

 • 0 PROTECTION_NONE                                                                                               
 • 1 PROTECTION_ALWAYS                                                                                             
 • 2 PROTECTION_LBD                                                                                                

2. cp_model_probing_level Defines the intensity of probing during presolve, where variables are temporarily fixed  
to infer more information about the problem. Higher levels of probing can result in a more simplified problem but  
require more computation time during presolve.                                                                     

3. ignore_subsolvers Specifies a list of subsolvers to exclude from use during the search. Removing certain        
subsolvers can free up resources and potentially speed up the search. However, excluding subsolvers can be risky as
it might eliminate strategies that are useful for solving difficult instances.                                     

From the OR-Tools documentation:                                                                                   

 • default_lp           (linearization_level:1)                                                                    
 • fixed                (only if fixed search specified or scheduling)                                             
 • no_lp                (linearization_level:0)                                                                    
 • max_lp               (linearization_level:2)                                                                    
 • pseudo_costs         (only if objective, change search heuristic)                                               
 • reduced_costs        (only if objective, change search heuristic)                                               
 • quick_restart        (kind of probing)                                                                          
 • quick_restart_no_lp  (kind of probing with linearization_level:0)                                               
 • lb_tree_search       (to improve lower bound, MIP like tree search)                                             
 • probing              (continuous probing and shaving)                                                           

4. search_branching Defines the branching strategy the solver uses to navigate the search tree. The options are:   

 • 0 (AUTOMATIC_SEARCH): The solver automatically selects the most appropriate strategy.                           
 • 1 (FIXED_SEARCH): Follows a fixed variable order, as specified by the user or the problem model.                
 • 2 (PORTFOLIO_SEARCH): Uses a combination of multiple strategies to explore the search space.                    
 • 3 (LP_SEARCH): Branches based on the LP relaxation of the problem, leveraging the reduced costs of variables.   
 • 4 (PSEUDO_COST_SEARCH): Branches using pseudo-costs, which are estimates of the impact of branching decisions   
   based on past experiences.                                                                                      
 • 5 (PORTFOLIO_WITH_QUICK_RESTART_SEARCH): Quickly explores different heuristics with low conflict limits, aiming 
   to find a good initial solution.                                                                                
 • 6 (HINT_SEARCH): Prioritizes decisions based on hints provided by the user or the problem model.                
 • 7 (PARTIAL_FIXED_SEARCH): Begins with a fixed strategy, then switches to automatic search for the remaining     
   decisions.                                                                                                      
 • 8 (RANDOMIZED_SEARCH): Introduces randomization into branching decisions to diversify the search.               

5. repair_hint Enables the solver to attempt to 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Metric                 ┃     Mean ┃      Min ┃      Max ┃ #Samples ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ Default Metric Value   │ 131935.0 │ 130872.0 │ 132805.0 │       10 │
│ Optimized Metric Value │ 131260.6 │ 128112.0 │ 132805.0 │       10 │
└────────────────────────┴──────────┴──────────┴──────────┴──────────┘

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────────────── WARNING ────────────────────────────────────────────────────╮
│                                                                                                                 │
│      The optimized parameters listed above were obtained based on a sampling approach                           │
│      and may not fully capture the complexities of the entire problem space.                                    │
│      While statistical reasoning has been applied, these results should be considered                           │
│      as a suggestion for further evaluation rather than definitive settings.                                    │
│                                                                                                                 │
│      It is strongly recommended to validate these parameters in larger, more comprehensive                      │
│      experiments before adopting them in critical applications.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯